In [ ]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [ ]:
!pip install transformers
# !pip install pytorch_lightning==1.2.10
!pip install pytorch_lightning

In [ ]:
!pip install datsets transformers[sentencepiece]
!pip install sentencepiece

In [24]:
print(pl.__version__)

2.0.2


In [3]:
# 라이브러리 불러오기

import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
def set_seed(seed): # 랜덤 적용
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

### 모델링
- 파이토치 라이트닝을 사용한다.

In [28]:
class T5FineTuner(pl.LightningModule):
    # def __init__(self, hparams):
    #     super(T5FineTuner, self).__init__() #  모델과 토크나이저를 불러옵니다.
    #     self.hparams = hparams
    # def __init__(self, **hparams):
    #     super(T5FineTuner, self).__init__() #  모델과 토크나이저를 불러옵니다.
    #     self.save_hyperparameters()
    def __init__(self, hparams):
        super(T5FineTuner, self).__init__()
        self.save_hyperparameters(hparams)

        self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)
        self.tokenizer = T5Tokenizer.from_pretrained(hparams.tokenizer_name_or_path)

    def is_logger(self): # 현재 모델 학습을 수행하는 프로세스의 랭크(rank)
        return self.trainer.proc_rank <= 0 
        # 첫 번째 프로세스에서만 로그를 출력하도록 하여 로깅을 중복되지 않도록 하는데 사용된다.

    def forward(
        self, input_ids, attention_mask = None, decoder_input_ids = None, decoder_attention_mask = None,
        lm_labels = None): # 순전파

        return self.model(input_ids, 
                          attention_mask = attention_mask, 
                          decoder_input_ids = decoder_input_ids,
                          decoder_attention_mask = decoder_attention_mask, 
                          lm_labels = lm_labels)
        
    def _step(self, batch): # 모델에 입력을 주고 손실을 계산한 후 손실값을 반환
        lm_labels = batch['target_ids'] # 입력 데이터에서 생성된 타겟 시퀀스를 저장
        lm_labels[lm_labels[:, :] == self.tokenizer.pad_token_id] = -100 # 모델이 패딩 토큰을 예측하지 못하도록 막는다.

        outputs = self(input_ids = batch['source_ids'],
                    attention_mask = batch['source_mask'],
                    lm_labels = lm_labels,
                    decoder_attention_mask=batch['target_mask'])
        
        loss = outputs[0]

        return loss # 손실값 반환

    def training_step(self, batch, batch_idx):
        loss = self._step(batch)

        tensorboard_logs = {'train_loss' : loss} # 학습 단계에서의 손실값을 계산합니다. 
        return {'loss' : loss, 'log' : tensorboard_logs} # tensorboard에 로그를 기록하고 손실값을 반환합니다.

    def training_epoch_end(self, outputs): # epoch마다 계산된 손실값의 평균값을 반환한다.
        avg_train_loss = torch.stack([x['loss'] for x in outputs]).mean()
        tensorboard_logs = {'avg_train_loss' : avg_train_loss}

    def validation_step(self, batch, batch_idx): #  검증 단계에서의 손실값을 계산합니다.
        loss = self._step(batch)
        return {'val_loss' : loss}

    def validation_epoch_end(self, outputs): # epoch마다 계산된 검증 손실값의 평균값을 반환한다.
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss' : avg_loss}
        return {'avg_val_loss' : avg_loss, 'log' : tensorboard_logs, 'progress_bar' : tensorboard_logs}

    def configure_optimizers(self):
        # 옵티마이저와 스케줄러를 정의한다.

        model = self.model
        no_decay = ['bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params' : [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
             'weight_decay' : self.hparams.weight_decay}, # 가중치 감쇠 적용

             {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
             "weight_decay": 0.0} # 이 그룹에 속한 파라미터는 가중치 감쇠를 적용하지 않는다
        ] # 각 파라미터 그업에 대해 서로 다른 가중치 감쇠를 적용한다.

        optimizer = AdamW(optimizer_grouped_parameters, lr = self.hparams.leraning_rate, eps = self.hparams.adam_epsilon)
        self.opt = optimizer
        return [optimizer]

    def get_tqdm_dict(self): #  tqdm에 표시할 딕셔너리를 반환합니다.
        tqdm_dict = {'loss' : '{:.3f}'.format(self.trainer.avg_loss), 'lr' : self.lr_schedulers.get_last_lr()[-1]}
        # 평균 손실 값을 소수점 세 자리까지 표시하고, 현재의 학습률을 표시

        return tqdm_dict

    def train_dataloader(self): # 학습 데이터 정의
        train_dataset = get_dataset(tokenizer=self.tokenizer, type_path="train", args=self.hparams)
        dataloader = DataLoader(train_dataset, batch_size = self.hparams.train_batch_size, drop_last=True, shuffle=True, num_workers=4)

        t_total = (
            (len(dataloader.dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
            // self.hparams.gradient_accumulation_steps * float(self.hparams.num_train_epoch)
        )

        scheduler = get_linear_schedule_with_warmup(
            self.opt, num_warmup_steps = self.hparams.warmup_steps, num_training_steps = t_total
        ) # get_linear_schedule_with_warmup : 옵티마이저와 학습률을 설정한다.
          # 초기에 0에서 시작하여 지정된 에포크마다 선형적으로 증가하고 단계를 거친 후 다시 선형적으로 감소시키는 방식으로 학습률을 조정한다.
          # num_warmup_steps : warmup 단계의 epoch 
          # num_training_steps : 전체 학습을 마칠 때까지 optimizer가 업데이트되는 횟수
        self.lr_schedulers = scheduler
        return dataloader

    def val_dataloader(self): # 검증 데이터 정의
        val_dataset = get_dataset(tokenizer=self.tokenizer, type_path="val", args=self.hparams)
        return DataLoader(val_dataset, batch_size = self.hparams.eval_batch_size, num_workers = 4)

In [6]:
logger = logging.getLogger(__name__)

# on_validation_end, on_test_end 오버라이드

class LoggingCallback(pl.Callback): # 학습 중 결과를 로깅하고 파일에 저장한다.
    def on_validation_end(self, trainer, pl_module):
        logger.info('-- Validation results --')
        if pl_module.is_logger():
            metrics = trainer.callback_metrics

            # 로그 결과
            for key in sorted(metrics):
                if key not in ['log', 'pregress_bar']:
                    logger.info('{} = {}\n'.format(key, str(metrics[key])))

    def on_test_end(self, trainer, pl_module):
        logger.info('-- Test results --')

        if pl_module.is_logger():
            metrics = trainer.callback_metrics

            # 로그외 결과를 파일로 저장한다.
            output_test_results_file = os.path.join(pl_module.hparams.output_dir, "test_results.txt")
            with open(output_test_results_file, "w") as writer:
                for key in sorted(metrics):
                    if key not in ["log", "progress_bar"]:
                        logger.info("{} = {}\n".format(key, str(metrics[key])))
                        writer.write("{} = {}\n".format(key, str(metrics[key])))

### 하이퍼 매개변수 및 기타 인수를 정의해 보겠습니다. 필요에 따라 특정 작업에 대해 이 사전을 재정의할 수 있습니다. 대부분의 경우 data_dir 및 output_dir만 변경하면 됩니다.

### 여기서 배치 크기는 8이고 gradient_accumulation_steps는 16이므로 효과적인 배치 크기는 128입니다.

In [7]:
args_dict = dict(
    data_dir = '', # 데이터 경로
    output_dir = '', # 체크포인트 경로
    model_name_or_path = 't5-base',
    tokenizer_name_or_path = 't5-base',
    max_seq_length = 512, # 입력 시퀀스의 최대 길이
    learning_rate = 3e-4, # 학습률
    weight_decay = 0.0, # L2 가중치 감쇠를 위한 파라미터
    adam_epsilon = 1e-8, # Adam optimizer에서 epsilon 값
    warmup_steps = 0, # learning rate scheduler에서 warmup steps
    train_batch_size = 8, # 훈련 배치 사이즈
    eval_train_size = 8, # 평가 배치 사이즈
    num_train_epochs = 2, # 에포크 수
    gradient_accumulation_steps = 16, # accumulation_steps의 수
    n_gpu = 1, # 사용할 gpu 수
    early_stop_callback = False, # early stopping 여부
    fp_16 = False, # mixed precision training 사용 여부
    opt_level = '01', # mixed precision training 시 사용할 최적화 레벨
    max_grad_norm = 1.0, # gradient clipping을 위한 최대 그래디언트 norm 값
    seed = 42 # random seed 값
)

In [ ]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xvf aclImdb_v1.tar.gz

In [9]:
train_pos_files = glob.glob('aclImdb/train/pos/*.txt')
train_neg_files = glob.glob('aclImdb/train/neg/*.txt')

In [10]:
len(train_pos_files), len(train_neg_files)

(12500, 12500)

### 검증을 위하여 훈련 데이터 셋에서 2000개를 추출하고 긍정/부정 리뷰를 각 1000개씩 val 디렉토리에 저장한다.

In [11]:
!mkdir aclImdb/val aclImdb/val/pos aclImdb/val/neg

In [12]:
random.shuffle(train_pos_files)
random.shuffle(train_neg_files)

val_pos_files = train_pos_files[:1000]
val_neg_files = train_neg_files[:1000]

In [13]:
import shutil

In [14]:
# 파일 이동

for f in val_pos_files: 
  shutil.move(f,  'aclImdb/val/pos')
for f in val_neg_files:
  shutil.move(f,  'aclImdb/val/neg')

### 데이터셋 준비

In [15]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [16]:
# 감정 토큰화

ids_neg = tokenizer.encode('negative </s>')
ids_pos = tokenizer.encode('positive </s>')
print(len(ids_neg), len(ids_pos))

2 2


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:226: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


### 리뷰가 긍정적이라면 긍정적, 부정적이라면 부정적으로 인코딩된다.

### html 태그를 제거하여 리뷰 텍스트를 정리한다. 또한 T5 모델에서 요구하는 입력 및 대상 끝에 eos 토큰 </s>를 추가한다.

In [17]:
class ImdbDataset(Dataset): # 리뷰 데이터셋을 처리하는 데이터셋 클래스
    def __init__(self, tokenizer, data_dir, type_path, max_len = 512): # () 안에 인스턴스 변수 초기화
        self.pos_file_path = os.path.join(data_dir, type_path, 'pos')
        self.neg_file_path = os.path.join(data_dir, type_path, 'neg')

        self.pos_files = glob.glob("%s/*.txt" % self.pos_file_path)
        self.neg_files = glob.glob("%s/*.txt" % self.neg_file_path)

        self.max_len = max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []

        self._build()

    def __len__(self): # inputs 길이 반환
        return len(self.inputs)

    def __getitem__(self, index): # source_ids, source_mask, target_ids, target_mask를 딕셔너리 형태로 반환
    # source_ids와 target_ids는 squeeze를 사용하여 1차원으로 변경
        source_ids = self.inputs[index]['input_ids'].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        src_mask = self.inputs[index]['attention_mask'].squeeze()
        target_mask = self.targets[index]['attention_mask'].squeeze()

        return {'source_ids' : source_ids, 'source_mask' : src_mask, 'target_ids' : target_ids, 'target_mask' : target_mask}

    def _build(self): # 해당 폴더에서 파일을 가져와 self._buil_examples_from_files 메서드 호출
        self._build_examples_from_files(self.pos_files, 'positive')
        self._build_examples_from_files(self.neg_files, 'negative')

    def _build_examples_from_files(self, files, sentiment): # 파일을 받아와 전처리 적용
        REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
        REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

        for path in files:
            with open(path, 'r') as f:
                text = f.read()

            line = text.strip()
            line = REPLACE_NO_SPACE.sub('', line)
            line = REPLACE_WITH_SPACE.sub('', line)
            line = line + ' </s>'

            target = sentiment + " </s>"

            # tokenize inputs
            tokenized_inputs = self.tokenizer.batch_encode_plus(
                [line], max_length = self.max_len, pad_to_max_length = True, return_tensors = 'pt'
            )

            # tokenize targets
            tokenized_targets = self.tokenizer.batch_encode_plus(
                [target], max_length = 2, pad_to_max_length = True, return_tensors = 'pt'
            ) # max_length가 2인 이유는 감성 레이블을 대표하는 토큰이기 때문이다.

            self.inputs.append(tokenized_inputs)
            self.targets.append(tokenized_targets)

In [18]:
dataset = ImdbDataset(tokenizer, 'aclImdb', 'val',  max_len=512)
len(dataset)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


2000

In [19]:
data = dataset[28]
print(tokenizer.decode(data['source_ids']))
print(tokenizer.decode(data['target_ids']))

Frailty is a nongory horror film that achieves its chills by following the logic and impact of a mans delusionobsession straight into depravity Dad we never learn his name is a gentle man and loving father whos raising his sons alone after Mom died giving birth to the youngest son Adam The familys world flips upside down late one night when Dad rushes into the boys room and tells them God has given him a vision And what a vision <unk> the entire familys job is to destroy demons who of course are disguised in human formProceeding from this premise the movie is unflinching in following it Dad kidnaps peopledemons whom God has told him to destroy binds them lays his hand on them to see a vision of their evil then kills them <unk> while making his young sons watch Fenton the older boy is horrified seeing only a father whos turned into a crazed murderer Adam the younger is uncomfortable but trusts that Dad is following Gods will Eventually Dad takes his sons on missions to abduct the demons

### 훈련

In [20]:
!mkdir -p t5_imdb_sentiment # 디렉토리 생성

### PyTorch Lightning 2.0.2를 사용함에 따라 pl.callbacks.ModelCheckpoint에 keyword argument가 조금 변경되었다.

In [51]:
# 입력된 인자를 args.dict에 저장
args_dict.update({'data_dir': 'aclImdb', 'output_dir': 't5_imdb_sentiment', 'num_train_epochs' : 2})
args = argparse.Namespace(**args_dict) # args 변수에 저장


# checkpoint_callback = pl.callbacks.ModelCheckpoint(
#     filepath=args.output_dir, prefix="checkpoint", monitor="val_loss", mode="min", save_top_k=5
# ) 원본

# 수정된 코드
# 학습 과정에서 모델의 가중치를 저장
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath = args.output_dir, save_last = 'checkpoint', monitor = 'val_loss', mode = 'min', save_top_k = 5)

# 토치 라이트닝 2.0.2에서 gpus를 accelerator로 변경해주어야 합니다.
# early_stop_callback을 early_stopping_callback으로 수정해주어야 합니다.
train_params = dict(
    accumulate_grad_batches = args.gradient_accumulation_steps,
    accelerator = args.n_gpu, # 수정한 부분 - 1
    max_epochs = args.num_train_epochs,
    early_stop_callback = False,
    precision = 16 if args.fp_16 else 32,
    amp_level = args.opt_level,
    gradient_clip_val = args.max_grad_norm,
    checkpoint_callback=checkpoint_callback,
    callbacks = [LoggingCallback()])

ModuleNotFoundError: ignored

### 데이터 세트를 반환하도록 get_dataset 함수를 정의합니다. 모델은 이 함수를 호출하여 train 및 val 데이터 세트를 가져옵니다

### 모델 코드를 전혀 수정할 필요가 없도록 데이터 세트 함수를 정의하고 있습니다. 문제에 따라 다른 데이터 세트를 반환하도록 함수를 재정의합니다. 이것이 현재 최상의 솔루션은 아니지만 작동합니다.

In [42]:
def get_dataset(tokenizer, type_path, args):
    return ImdbDataset(tokenizer=tokenizer, data_dir=args.data_dir, type_path=type_path,  max_len=args.max_seq_length)

### 모델 초기화

### AttributeError: can't set attribute 'hparams'문제에 대응하기 위하여 스택 오버 플로우를 참고하여 버젼을 바꿔보기로한다.

- 결과는 실패하였다. 그래서 self.hparams를 수정하였다.

In [49]:
model = T5FineTuner(hparams=args)

trainer 초기화

In [50]:
trainer = pl.Trainer(**train_params, auto_lr_find = False)

TypeError: ignored